In [2]:
!pip install pgmpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 4.9 MB/s 


In [1]:
# Import useful packages
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.inference import VariableElimination
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [4]:
# If using colab
# Upload file from location
from google.colab import files

uploaded = files.upload()

Saving weatherHistory.csv to weatherHistory.csv


In [2]:
# If not using colab
# Load datas from files and filter the data to use
def read_data(filepath):
    data = pd.read_csv(filepath)
    return data

data = read_data("weatherHistory.csv")
data = data[["Formatted Date", "Summary","Precip Type", "Temperature (C)", "Humidity", "Wind Speed (km/h)", "Visibility (km)"]].sort_values(by=['Formatted Date'])
data['Precip Type'].fillna("rain", inplace = True)

In [8]:
# Encoder labels and numeriser the data
Encoder = LabelEncoder()
data['Precip Type']=Encoder.fit_transform(data['Precip Type'])
data['Summary']=Encoder.fit_transform(data['Summary'])
data = data.drop(columns=['Formatted Date'])

Temperature = data['Temperature (C)'].apply(lambda x:1 if x>=5 else 0).to_frame()
Humidity = data['Humidity'].apply(lambda x:1 if x>=0.5 else 0).to_frame()
WindSpeed = data['Wind Speed (km/h)'].apply(lambda x:0 if x<=30 else 1).to_frame()
Visibility = data['Visibility (km)'].apply(lambda x:0 if x<=8 else 1).to_frame()

data_new = pd.concat([data.drop(columns=['Temperature (C)']), Temperature],axis=1)
data_new = pd.concat([data_new.drop(columns=['Humidity']), Humidity],axis=1)
data_new = pd.concat([data_new.drop(columns=['Wind Speed (km/h)']), WindSpeed],axis=1)
data_new = pd.concat([data_new.drop(columns=['Visibility (km)']), Visibility],axis=1)

In [4]:
# Define a BayesianNetwork with nodes we get from csv
# , ("Formatted Date", "Summary")
model = BayesianNetwork([("Precip Type", "Summary"), ("Temperature (C)", "Summary"), ("Humidity", "Summary"), ("Wind Speed (km/h)", "Visibility (km)"), ("Visibility (km)", "Summary")])

In [16]:
# Train model with the data w've preprocessed, we choose one year to train
data_train = data_new.iloc[0:17522,:]
model.fit(data_train)
model.get_cpds()

[<TabularCPD representing P(Precip Type:2) at 0x7f3bd180e110>,
 <TabularCPD representing P(Summary:19 | Humidity:2, Precip Type:2, Temperature (C):2, Visibility (km):2) at 0x7f3bdf885250>,
 <TabularCPD representing P(Temperature (C):2) at 0x7f3bd181ba10>,
 <TabularCPD representing P(Humidity:2) at 0x7f3bd20886d0>,
 <TabularCPD representing P(Wind Speed (km/h):2) at 0x7f3bd41a15d0>,
 <TabularCPD representing P(Visibility (km):2 | Wind Speed (km/h):2) at 0x7f3bd4426890>]

In [29]:
data_test = data_new.iloc[144:168,:].drop(['Summary'],axis=1)
model.predict(data_test)

  0%|          | 0/4 [00:00<?, ?it/s]

,Summary
0,12
1,12
2,12
3,12
4,12
5,12
6,12
7,12
8,12
9,12
